## Task 2.1. LLM-Assisted Meta-Objective Definition

**Purpose:** To assist the Operative in distilling a project's core purpose into a set of compelling Meta-Objective statements, formatted for easy inclusion in input_ops.json.

**User Inputs to LLM:**
* Project Name/Concept
* Raw Project Description

You are an expert project analyst working with Operative Kin-Caid of the Chiron Guild.

**Your Current Task:** Meta-Objective Definition.
Your goal is to assist Operative Kin-Caid in drafting 3-5 concise, compelling Meta-Objective statements based on the provided project information.

**Process:**
1.  You will be provided with a Project Name/Concept and a Raw Project Description.
2.  Based on this input, draft 3-5 Meta-Objective statements.
3.  Each statement should be a single sentence capturing the absolute core purpose and primary value proposition of the project for the Chiron Guild or the specific Operative endeavor.
4.  Focus on the ultimate 'win' or desired future state.
5.  **Output Format:** Structure your response as a **JSON array**. Each object in the array should represent one Meta-Objective and MUST contain the following keys:
    *   `"id"` (string): A short identifier (e.g., "M1", "M2", "M3").
    *   `"text"` (string): The concise Meta-Objective statement.



In [ ]:
import os
import json
import google.generativeai as genai
from pathlib import Path

# Configure Gemini (reads from Codespace secret)
genai.configure(api_key=os.environ.get("GEMINI_API_KEY"))

# Define model configuration (match generate_briefs_for_review.py)
GEMINI_MODEL_NAME = "gemini-2.5-flash-preview-05-20" 
generation_config = {
    "temperature": 0.7,
    "top_p": 1,
    "top_k": 1,
    "max_output_tokens": 8192,
    "response_mime_type": "application/json",
}
safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
]

# --- INPUTS FOR THIS TASK ---
project_name_concept = "Chiron Guild Internal Communications Hub"
raw_project_description = """
The Chiron Guild currently uses a mix of Discord, email, and shared documents for internal communication. This is becoming fragmented and inefficient as we grow. We need a centralized, self-hosted platform that integrates chat, knowledge base, and announcements. The platform should be secure, extensible, and align with our values of data sovereignty and transparency. The goal is to improve information flow, reduce time spent searching for information, and foster a stronger sense of community among Chironians. This will ultimately make our cooperative more effective and resilient.
"""

# --- CONSTRUCT THE PROMPT ---
# You'll paste the LLM Prompt from project_decomposition.md (Task 2.1) here.
# Replace the dynamic parts with your actual inputs.
llm_prompt_template_str = """
You are an expert project analyst working with Operative Kin-Caid of the Chiron Guild.

**Your Current Task:** Meta-Objective Definition.
Your goal is to assist Operative Kin-Caid in drafting 3-5 concise, compelling Meta-Objective statements based on the provided project information.

**Process:**
1.  You will be provided with a Project Name/Concept and a Raw Project Description.
2.  Based on this input, draft 3-5 Meta-Objective statements.
3.  Each statement should be a single sentence capturing the absolute core purpose and primary value proposition of the project for the Chiron Guild or the specific Operative endeavor.
4.  Focus on the ultimate 'win' or desired future state.
5.  **Output Format:** Structure your response as a **JSON array**. Each object in the array should represent one Meta-Objective and MUST contain the following keys:
    *   `"id"` (string): A short identifier (e.g., "M1", "M2", "M3").
    *   `"text"` (string): The concise Meta-Objective statement.

**Chiron Guild Context Reminder:**
The Chiron Guild is a worker-owned, AI-augmented digital cooperative with a 'Mythic Core, Precision Shell' ethos. Your analysis should align with this mission.
"""

# Combine LLM system instruction with user input
# The actual prompt to Gemini will be simpler, just the request.
# The "User Input" section from project_decomposition.md describes what *you* provide.
# The "LLM Prompt" section from project_decomposition.md is the *system instruction* for Gemini.

# For interactive use, the best way to handle this is a multi-turn approach or
# concatenate the system instruction with the user's specific request.

# Let's create a single prompt that Gemini can respond to based on task 2.1 description
full_prompt_for_gemini = f"""
{llm_prompt_template_str}

---
Project Name/Concept: {project_name_concept}
Raw Project Description: {raw_project_description}

Task: Generate the JSON array of Meta-Objectives.
"""

# --- CALL GEMINI API ---
model = genai.GenerativeModel(
    model_name=GEMINI_MODEL_NAME,
    generation_config=generation_config,
    safety_settings=safety_settings
)

print(f"Calling Gemini model: {GEMINI_MODEL_NAME} for Meta-Objective Definition...")
try:
    response = model.generate_content(full_prompt_for_gemini)
    
    if response.text:
        meta_objectives_json = json.loads(response.text)
        print("\n--- Generated Meta-Objectives (JSON) ---")
        print(json.dumps(meta_objectives_json, indent=2))
        # You can now save this to a file or a variable for next steps
        with open("temp_meta_objectives.json", "w") as f:
            json.dump(meta_objectives_json, f, indent=2)
        print("\nSaved to temp_meta_objectives.json for review.")

    else:
        print("ERROR: Gemini API response was empty or did not contain text content.")
        print(f"Full Gemini Response object: {response}")

except Exception as e:
    print(f"ERROR during Gemini API call: {e}")
    # Print full response for debugging if it's not text
    if hasattr(response, 'prompt_feedback') and response.prompt_feedback.block_reason:
        print(f"Gemini API blocked response due to: {response.prompt_feedback.block_reason}")